<a href="https://colab.research.google.com/github/abdulwahab724/adChatbot/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Set visual style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [ ]:
df = pd.read_excel('Dataset.xlsx', sheet_name='User_Stories')

print("="*80)
print("📊 DATASET LOADED")
print("="*80)
print(f"Total records: {df.shape[0]:,}")
print(f"Total columns: {df.shape[1]}")
print(f"\nFirst few rows:")
display(df.head(3))

📊 DATASET LOADED
Total records: 12,847
Total columns: 24

First few rows:


,StoryID,CompanyID,ProjectID,StoryText,WordCount,CreationDate,Domain,CompanySize,AgileExperience,TeamSize,...,ActorAmbiguity,AcceptanceAmbiguity,DependencyAmbiguity,PriorityAmbiguity,TechnicalAmbiguity,DefectCount,DevelopmentTime,StoryPoints,CustomerSatisfaction,CompletionStatus
0,US-C1-001,C1,P1,"As a loan officer, I want to process applicati...",55,2023-04-07,Finance,Large,7,8,...,False,False,False,False,False,1,7,13,3.2,Done
1,US-C1-002,C1,P1,"As a customer, I want to process application i...",36,2023-04-07,Finance,Large,7,6,...,False,False,False,False,False,2,5,5,3.5,In Progress
2,US-C1-003,C1,P1,"As a user, I would like to view account balanc...",61,2023-04-07,Finance,Large,7,7,...,True,False,False,False,False,2,8,1,3.4,Done


In [ ]:
# ========================================
# STEP 2: Keep only necessary columns
# ========================================
columns_to_keep = [
    'StoryText',           # Input text for LLM
    'SemanticAmbiguity',   # Target labels
    'ScopeAmbiguity',
    'ActorAmbiguity',
    'AcceptanceAmbiguity',
    'DependencyAmbiguity',
    'PriorityAmbiguity',
    'TechnicalAmbiguity'
]

df_cleaned = df[columns_to_keep].copy()

print("="*80)
print("✅ STEP 2: COLUMNS FILTERED")
print("="*80)
print(f"Original: {df.shape[1]} columns")
print(f"Cleaned: {df_cleaned.shape[1]} columns")
print(f"Removed: {df.shape[1] - df_cleaned.shape[1]} columns")
print(f"\nKept columns: {df_cleaned.columns.tolist()}")

✅ STEP 2: COLUMNS FILTERED
Original: 24 columns
Cleaned: 8 columns
Removed: 16 columns

Kept columns: ['StoryText', 'SemanticAmbiguity', 'ScopeAmbiguity', 'ActorAmbiguity', 'AcceptanceAmbiguity', 'DependencyAmbiguity', 'PriorityAmbiguity', 'TechnicalAmbiguity']


In [ ]:
# ========================================
# STEP 5: Convert boolean columns to text
# ========================================
print("="*80)
print("✅ STEP 5: CONVERTING BOOLEANS TO TEXT")
print("="*80)

def convert_ambiguities_to_text(row):
    """Convert True/False columns into readable text"""

    ambiguity_map = {
        'SemanticAmbiguity': 'Semantic Ambiguity',
        'ScopeAmbiguity': 'Scope Ambiguity',
        'ActorAmbiguity': 'Actor Ambiguity',
        'AcceptanceAmbiguity': 'Acceptance Ambiguity',
        'DependencyAmbiguity': 'Dependency Ambiguity',
        'PriorityAmbiguity': 'Priority Ambiguity',
        'TechnicalAmbiguity': 'Technical Ambiguity'
    }

    # Find which ambiguities are True
    found = []
    for col, label in ambiguity_map.items():
        if row[col] == True:
            found.append(label)

    # Return result
    if len(found) == 0:
        return "No ambiguities"
    else:
        return ", ".join(found)

# Apply conversion
df_cleaned['Ambiguities'] = df_cleaned.apply(convert_ambiguities_to_text, axis=1)

print("Conversion complete!")
print(f"\nSample conversions:")
print(df_cleaned[['StoryText', 'Ambiguities']].head(10).to_string(index=False))

✅ STEP 5: CONVERTING BOOLEANS TO TEXT
Conversion complete!

Sample conversions:
                                                                                                                               StoryText                                          Ambiguities
                                                    As a loan officer, I want to process application in order to comply with regulations                                   Semantic Ambiguity
                                                                      As a customer, I want to process application in order to save time                                   Semantic Ambiguity
  As a user, I would like to view account balance to comply with regulations with detailed specifications and comprehensive requirements                                      Actor Ambiguity
                                                   As a financial advisor, I want to access system so that I can comply with regulations                        

In [ ]:
# ========================================
# STEP 6: Remove boolean columns
# ========================================
print("\n" + "="*80)
print("✅ STEP 6: REMOVING BOOLEAN COLUMNS")
print("="*80)

# Drop individual boolean columns (we have 'Ambiguities' now)
boolean_cols = [
    'SemanticAmbiguity', 'ScopeAmbiguity', 'ActorAmbiguity',
    'AcceptanceAmbiguity', 'DependencyAmbiguity',
    'PriorityAmbiguity', 'TechnicalAmbiguity'
]

df_final = df_cleaned.drop(columns=boolean_cols)

print(f"Removed {len(boolean_cols)} boolean columns")
print(f"\nFinal columns: {df_final.columns.tolist()}")
print(f"Final shape: {df_final.shape}")


✅ STEP 6: REMOVING BOOLEAN COLUMNS
Removed 7 boolean columns

Final columns: ['StoryText', 'Ambiguities']
Final shape: (12847, 2)


In [ ]:
# ========================================
# STEP 8: Balance the dataset
# ========================================
print("\n" + "="*80)
print("✅ STEP 8: BALANCING DATASET")
print("="*80)

# Separate stories with and without ambiguities
no_ambiguity = df_final[df_final['Ambiguities'] == 'No ambiguities']
with_ambiguity = df_final[df_final['Ambiguities'] != 'No ambiguities']

print(f"Before balancing:")
print(f"   No ambiguities: {len(no_ambiguity):,}")
print(f"   With ambiguities: {len(with_ambiguity):,}")

# Balance by undersampling the majority class
min_count = min(len(no_ambiguity), len(with_ambiguity))

no_ambiguity_balanced = no_ambiguity.sample(n=min_count, random_state=42)
with_ambiguity_balanced = with_ambiguity.sample(n=min_count, random_state=42)

# Combine
df_balanced = pd.concat([no_ambiguity_balanced, with_ambiguity_balanced])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle

print(f"\nAfter balancing:")
print(f"   No ambiguities: {len(no_ambiguity_balanced):,}")
print(f"   With ambiguities: {len(with_ambiguity_balanced):,}")
print(f"   Total: {len(df_balanced):,}")
print(f"\n✅ Dataset is now balanced!")


✅ STEP 8: BALANCING DATASET
Before balancing:
   No ambiguities: 8,520
   With ambiguities: 4,327

After balancing:
   No ambiguities: 4,327
   With ambiguities: 4,327
   Total: 8,654

✅ Dataset is now balanced!


In [ ]:
# ========================================
# STEP 9: Save preprocessed data
# ========================================
print("\n" + "="*80)
print("✅ STEP 9: SAVING PREPROCESSED DATA")
print("="*80)

# Save to CSV
df_balanced.to_csv('preprocessed_dataset.csv', index=False)
print(f"✅ Saved as 'preprocessed_dataset.csv'")
print(f"   Records: {len(df_balanced):,}")
print(f"   Columns: {df_balanced.columns.tolist()}")

# Show final preview
print(f"\n📋 Final Dataset Preview:")
display(df_balanced.head(10))


✅ STEP 9: SAVING PREPROCESSED DATA
✅ Saved as 'preprocessed_dataset.csv'
   Records: 8,654
   Columns: ['StoryText', 'Ambiguities']

📋 Final Dataset Preview:


,StoryText,Ambiguities
0,"As a nurse, I want to access records in order ...",Semantic Ambiguity
1,"As a bank customer, I need to transfer funds s...",No ambiguities
2,"As a patient, I would like to handle patient c...",Semantic Ambiguity
3,"As a vendor, I would like to search products t...",No ambiguities
4,"As a buyer, I want to checkout in order to tra...",No ambiguities
5,"As a patient, I need to process medical inform...",Semantic Ambiguity
6,"As a vendor, I want to checkout so that I can ...",No ambiguities
7,"As a account holder, I need to transfer funds ...",No ambiguities
8,"As a customer, I want to transfer funds in ord...",No ambiguities
9,"As a buyer, I want to leave review so that I c...",No ambiguities
